In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于批量预测的AutoML表格分类模型，并附带解释

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Python的Vertex客户端库来创建表格分类模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测，并附带解释。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview)。 这个数据集不需要任何特征工程。 您在本教程中使用的数据集版本存储在公共云存储桶中。 训练好的模型可预测三种类型的鸢尾花物种：山鸢尾（setosa）、弗吉尼亚鸢尾（virginica）或三色堇（versicolor）。

### 目标

在本教程中，您将从 Python 脚本中创建一个 AutoML 表格分类模型，然后使用 Vertex 客户端库进行批量预测并解释结果。您也可以选择使用 `gcloud` 命令行工具或在 Google Cloud 控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测并进行解释。

使用批量预测和使用在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项目）进行即时预测并实时返回结果。
  
* 批量预测服务：对整个实例集进行后台排队（批量）预测，并在准备就绪时将结果存储在 Cloud Storage 存储桶中。

成本

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)来根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *云存储*，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果可以的话，请确保在 GPU 运行时下运行这个笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得 $300 的免费信用用于计算/存储成本。

2. [确保为您的项目启用了结算功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行这个单元格，以确保 Cloud SDK 对本笔记本中所有命令使用正确的项目。

**注意**：Jupyter 将以 `!` 开头的行作为 shell 命令运行，并将以 `$` 开头的 Python 变量插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来训练Vertex。并非所有地区都支持所有Vertex服务。有关每个地区的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行现场教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户之间在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将其附加到将在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Google Cloud Notebook**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并在提示时按照说明通过 oAuth 验证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到 [创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，选择**Vertex 管理员**。在筛选框中输入"Storage Object Admin"，选择**Storage Object Admin**。

点击创建。一个包含您密钥的 JSON 文件将下载到您的本地环境。

将您的服务帐户密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量输入到下面的单元格中，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用的是哪种笔记本环境，以下步骤都是必需的。**

本教程旨在使用存储在公共云存储桶中的训练数据，并为您的批量预测使用本地云存储桶。您也可以选择使用您自己存储在本地云存储桶中的训练数据。

在下面设置您的云存储桶的名称。云存储桶的名称必须在所有Google Cloud项目中全球唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶还不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

#### 导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

import google.cloud.aiplatform_v1beta1 as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点 API 服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

自动机器学习（AutoML）常量

设置专属于AutoML数据集和训练的常量：

- 数据集结构：告诉`数据集`资源服务数据集的类型。
- 数据标注（注释）结构：告诉`数据集`资源服务数据的标签（注释）是如何进行的。
- 数据集训练结构：告诉`管道`资源服务要为哪种任务（例如分类）训练模型。

In [ ]:
# Tabular Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
LABEL_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)
# Tabular Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

硬件加速器

为预测设置硬件加速器（如 GPU）。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像，以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个带有 4 个 Nvidia Telsa K80 GPU 分配给每个 VM 的 GPU 容器映像，您可以指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定 `(None, None)` 来使用一个可以在 CPU 上运行的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

容器（Docker）映像

对于AutoML批量预测，Vertex预测服务预先确定了用于服务二进制文件的容器映像。更具体地说，该服务将根据您选择的硬件加速器选择适当的模型容器。

#### 机器类型

接下来，设置要用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]中的数字

*注意：您也可以使用n2和e2机器类型进行培训和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您准备好开始创建自己的AutoML表格分类模型。

建立客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本）中，您将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 用于`Dataset`资源的数据集服务。
- 用于`Model`资源的模型服务。
- 用于训练的Pipeline服务。
- 用于批量预测和定制训练的作业服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户已经准备好了，您的第一步是创建一个`Dataset`资源实例。这一步与Vision、Video和Language有所不同。对于那些产品，在`Dataset`资源创建之后，然后单独使用`import_data`方法导入数据。

对于表格式数据，数据的导入被推迟到训练管道开始训练模型时。我们要做什么不同的呢？首先，您不会调用`import_data`方法。相反，当您创建数据集实例时，您需要在`Dataset`资源的元数据中指定包含您表格式数据的CSV文件的Cloud Storage位置或数据表的BigQuery位置。

#### Cloud Storage

`metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}`

Cloud Storage路径的格式为：

    gs://[bucket_name]/[folder(s)/[file]

#### BigQuery

`metadata = {"input_config": {"bigquery_source": {"uri": [gcs_uri]}}}`

BigQuery路径的格式为：

    bq://[collection].[dataset].[table]

请注意，`uri`字段是一个列表，您可以在数据分布在多个文件时输入多个CSV文件或BigQuery表。

### 数据准备

Vertex中关于表格数据的`Dataset`资源有一些要求。

- 必须是CSV文件或BigQuery查询。

CSV

对于表格分类，CSV 文件有一些要求：

- 第一行必须是标题 -- 请注意这与 Vision、Video 和 Language 不同，它的要求是没有标题。
- 除了一个列是特征外，其余列都是特征。
- 一个列是标签，您将在随后创建训练流程时指定标签。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的鸢尾花数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

您还需要知道训练时标签列的列名，它保存为`label_column`。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

## 数据集

现在您的客户准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建 `Dataset` 资源的实例。该函数执行以下操作：

1. 使用数据集客户服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的用于显示的名称。
 - `metadata_schema_uri`：数据集类型的模式。
 - `metadata`：表格数据的 Cloud Storage 或 BigQuery 位置。
3. 调用客户数据集服务方法 `create_dataset`，具有以下参数：
 - `parent`：`Database`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
 - `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理异步调用长时间运行操作的方式。虽然这一步通常很快，但当您在项目中首次使用它时，由于需进行预配，会有较长延迟。

您可以使用 `operation` 对象来获取操作的状态（例如，创建 `Dataset` 资源）或取消操作，通过调用操作方法：

| 方法      | 描述 |
| ----------- | ----------- |
| result()    | 等待操作完成并以 JSON 格式返回一个结果对象。      |
| running()   | 返回操作是否仍在运行的 True/False。        |
| done()      | 返回操作是否已完成的 True/False。 |
| canceled()  | 返回操作是否已取消的 True/False。 |
| cancel()    | 取消操作（可能需要最多 30 秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, src_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        if src_uri.startswith("gs://"):
            metadata = {"input_config": {"gcs_source": {"uri": [src_uri]}}}
        elif src_uri.startswith("bq://"):
            metadata = {"input_config": {"bigquery_source": {"uri": [src_uri]}}}
        dataset = aip.Dataset(
            display_name=name,
            metadata_schema_uri=schema,
            labels=labels,
            metadata=json_format.ParseDict(metadata, Value()),
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("iris-" + TIMESTAMP, DATA_SCHEMA, src_uri=IMPORT_FILE)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

## 训练模型

现在使用您的Vertex `Dataset`资源训练一个AutoML表格分类模型。要训练模型，请执行以下步骤：

1. 为`Dataset`资源创建一个Vertex训练管道。
2. 执行管道以开始训练。

### 创建训练管道

你可能会问，我们用管道做什么？通常在工作（比如训练）有多个步骤时，我们会使用管道，这些步骤通常按顺序排列：执行步骤A，执行步骤B，依此类推。通过将这些步骤放入管道中，我们可以获得以下好处：

1. 可以重复使用于后续的训练工作。
2. 可以将其容器化并作为批处理作业运行。
3. 可以分布式执行。
4. 所有步骤都关联到相同的管道作业以追踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：管道作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标注（注释）的训练模式。
- `task`：描述训练作业要求的字典。

辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，它接受以下参数：

- `parent`：为您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规范。

让我们现在更深入地了解构建 `training_pipeline` 规范的 *最低* 要求：

- `display_name`：管道作业的人类可读名称。
- `training_task_definition`：数据集标注（注释）的训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅为 Vertex 数据集标识符（非完全限定）--这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，则用于训练、测试和验证的数据集百分比。否则，百分比将由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，需要传入一个 Google protobuf 结构，而不是 Python（类似 JSON）字典的 `task` 字段。您可以使用 `json_format.ParseDict` 方法进行转换。

您需要指定的最小字段包括：

- `prediction_type`：我们是在进行“分类”还是“回归”任务。
- `target_column`：我们要预测的 CSV 标题列名（即标签）。
- `train_budget_milli_node_hours`：用于训练模型的最大时间预算（计费时间），其中 1000 = 1 小时。
- `disable_early_stopping`：True/False 是否允许 AutoML 根据其判断提前停止训练或继续训练至整个预算结束。
- `transformations`：为每个特征列指定特征工程。

对于 `transformations`，列表必须包含每个列的条目。外部键字段指示相应列的特征工程类型。在本教程中，将其设置为 `"auto"`，以告诉 AutoML 自动确定。

最后，调用辅助函数 `create_pipeline` 创建管道，该函数将返回一个训练管道对象的实例。

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "sepal_width"}},
    {"auto": {"column_name": "sepal_length"}},
    {"auto": {"column_name": "petal_length"}},
    {"auto": {"column_name": "petal_width"}},
]

In [ ]:
PIPE_NAME = "iris_pipe-" + TIMESTAMP
MODEL_NAME = "iris_model-" + TIMESTAMP

task = Value(
    struct_value=Struct(
        fields={
            "target_column": Value(string_value=label_column),
            "prediction_type": Value(string_value="classification"),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "disable_early_stopping": Value(bool_value=False),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取关于训练流水线的信息

现在只获取这个训练流水线实例的流水线信息。通过调用作业客户端服务的`get_training_pipeline`方法，并使用以下参数来获取该作业的作业信息：

- `name`：Vertex的完全限定流水线标识。

当模型训练完成时，流水线状态将会是`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练上述模型可能需要超过30分钟的时间。

一旦您的模型训练完成，您可以通过将“结束时间”减去“开始时间”来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的 Vertex 模型资源标识符，这是管道服务分配给它的。您可以从返回的管道实例中获取这个信息，作为字段“model_to_deploy.name”。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在您的模型已经训练完成，您可以获取一些关于您的模型的信息。

评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被留出作为测试（留出）数据，由流水线服务用于评估模型。

###列出所有切片的评估

使用这个辅助函数`list_model_evaluations`，它接受以下参数：

-`name`：`Model`资源的Vertex完全合格的模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是评估指标。

对于每个评估（您可能只有一个），然后我们打印出评估中每个指标的所有关键名称，并且对于一小组（`logLoss`和`auPrc`），您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 用于批量预测的模型部署

现在部署您为批量预测创建的已训练的 Vertex `Model` 资源。这与部署用于按需预测的 `Model` 资源不同。

对于在线预测，您需要：

1. 创建一个用于部署 `Model` 资源的 `Endpoint` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

3. 向 `Endpoint` 资源发送在线预测请求。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求提供资源。

3. 将批量预测请求的结果返回给调用者。

4. 作业服务将取消为批量预测请求提供的资源。

## 进行批量预测请求

现在对您部署的模型进行批量预测。

制作测试条目

您将使用合成数据作为测试数据项。不要担心我们使用合成数据 -- 我们只是想演示如何进行预测。

In [ ]:
HEADING = "petal_length,petal_width,sepal_length,sepal_width"
INSTANCE_1 = "1.4,1.3,5.1,2.8"
INSTANCE_2 = "1.5,1.2,4.7,2.4"

### 制作批量输入文件

现在制作一个批量输入文件，您将把它存储在您的本地云存储存储桶中。与图像、视频和文本不同，用于表格的批量输入文件只支持CSV。对于CSV文件，您要做的是：

- 第一行是带有特征（字段）标题名称的标题。
- 每个剩余的行是一个单独的预测请求，带有相应的特征值。

例如：

    "feature_1", "feature_2", ...
    value_1, value_2, ...

In [ ]:
import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.csv"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    f.write(HEADING + "\n")
    f.write(str(INSTANCE_1) + "\n")
    f.write(str(INSTANCE_2) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例的扩展

处理批量预测请求时，您有几种扩展计算实例的选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：批量预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例被部署，并且批量预测请求在它们之间均匀分布。

- 自动扩展：批量预测请求被分配到可扩展数量的计算实例上。
  - 设置计算实例的最小（`MIN_NODES`）数量，在模型首次部署时进行和取消部署，并设置最大（`MAX_NODES`）计算实例的数量，根据负载情况进行部署。

计算实例的最小数量对应于`min_replica_count`字段，最大计算实例数量对应于`max_replica_count`字段，在您之后的部署请求中使用。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 进行批量预测请求

现在你的两个测试项的批量已经准备好了，让我们进行批量请求。使用以下帮助函数`create_batch_prediction_job`，并传入以下参数：

- `display_name`：预测任务的可读名称。
- `model_name`：`Model`资源的 Vertex 完全限定标识符。
- `gcs_source_uri`：输入文件的 Cloud Storage 路径 —— 就是你之前创建的那个。
- `gcs_destination_output_uri_prefix`：服务将把预测结果写入的 Cloud Storage 路径。
- `parameters`：用于提供预测结果的额外过滤参数。

该帮助函数会调用作业客户端服务的 `create_batch_prediction_job` 方法，并传入以下参数：

- `parent`：Dataset、Model 和 Pipeline 资源的 Vertex 根路径。
- `batch_prediction_job`：批量预测任务的规范。

现在让我们深入了解 `batch_prediction_job` 的规范：

- `display_name`：批量预测任务的可读名称。
- `model`：`Model`资源的 Vertex 完全限定标识符。
- `dedicated_resources`：为批量预测任务提供的计算资源。
  - `machine_spec`：要提供的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `starting_replica_count`：最初提供的计算实例数，你之前设定为变量 `MIN_NODES`。
  - `max_replica_count`：可扩展到的最大计算实例数，你之前设定为变量 `MAX_NODES`。
- `model_parameters`：用于提供预测结果的额外过滤参数。*注意*，图像分割模型不支持额外参数。
- `input_config`：要进行预测的实例的输入来源和格式类型。
 - `instances_format`：批量预测请求文件的格式：仅支持 `csv`。
 - `gcs_source`：一到多个用于批量预测请求的 Cloud Storage 路径的列表。
- `output_config`：预测结果的输出目的地和格式。
 - `prediction_format`：批量预测响应文件的格式：仅支持 `csv`。
 - `gcs_destination`：预测结果的输出目的地。

该调用是一个异步操作。你将从响应对象中打印出一些选择字段，包括：

- `name`：分配给批量预测任务的 Vertex 完全限定标识符。
- `display_name`：批量预测任务的可读名称。
- `model`：Model 资源的 Vertex 完全限定标识符。
- `generate_explanations`：预测是否提供了 True/False 解释（可解释性）。
- `state`：预测任务的状态（等待中、运行中等）。

由于该调用需要一些时间来执行，你可能会在 `state` 中获得 `JobState.JOB_STATE_PENDING`。

In [ ]:
BATCH_MODEL = "iris_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
        "generate_explanation": True,
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "csv"
OUT_FORMAT = "csv"  # [csv]

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME, None
)

现在获取您创建的批量预测任务的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

获取有关批量预测作业的信息

使用该辅助函数`get_batch_prediction_job`，具有以下参数：

- `job_name`：批量预测作业的Vertex完全限定标识符。

辅助函数调用作业客户端服务的`get_batch_prediction_job`方法，具有以下参数：

- `name`：批量预测作业的Vertex完全限定标识符。在本教程中，您将传递批量预测作业的Vertex完全限定标识符--`batch_job_id`

辅助函数将返回存储预测结果的Cloud Storage路径--`gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取带有解释的预测结果

当批量预测处理完成时，作业状态将会是`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在您设置为输出的云存储路径中的预测结果和相应的解释。这些解释将采用CSV格式，您在创建批量解释作业时指定的，在以`prediction`开头的子文件夹下，该文件夹下将有一个名为`explanations*.csv`的文件。

现在显示（cat）这些内容。您将会看到每一条解释占据一行。

- 前四个字段是进行预测的值（特征）。
- 剩余字段是每个预测的置信度值，介于0和1之间。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/explanation*.csv

        ! gsutil cat $folder/explanation*.csv
        break
    time.sleep(60)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于教程的GCP项目。

否则，您可以删除此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME